# Create Catalog for CESM output

In [1]:
import pathlib
from os import listdir
from os.path import isfile, join
import shutil

import intake
from ecgtools import Builder
from ecgtools.parsers.cesm import parse_cesm_timeseries

import pandas as pd 
import xarray as xr
import numpy as np
import itertools

## Step 0: Move Files from `data_tmp` to `data` and Create New Folders if needed

In [2]:
# Path of new downloaded CESM data 
path='/data/keeling/a/cristi/a/data_tmp/cesm'
# rootpath of target directory.
rootpath='/data/keeling/a/cristi/a/esm_data/??'#Which directory should we use? 

def target_location_cmip(fname,rootpath): #generates target location 
    if rootpath[-1]!='/':
        rootpath=rootpath+'/'
    fname_set=fname.split('_')     
    location=rootpath+fname_set[2]+'/'+fname_set[3]+'/'+fname_set[4]+'/'# index need change based on CESM files are named
    return location

# move files from current folder to target folders
k_moved=0
for j in listdir(path): #read all downlaoded files in data_tmp
    current_dir=os.path.join(path, j) 
    target_dir= target_location_cmip(j,rootpath)
    if target_dir!=current_dir:
        if os.path.exists(target_dir)==False:
            os.makedirs(target_dir) #make directories 
        shutil.move(current_dir,target_dir) #move files 
        k_moved+=1
        
print('moved '+str(k_moved)+' files')

moved 0 files


## Step 1: Get updated list of Subdirectories

1. Log onto Keeling and go to `/data/keeling/a/cristi/a/esm_data/cesm`

2. Run the following line of code to the get the most updated list of all subdirectories

`readlink -f $(find . -type d -path '*/r*' -prune) > subdir_list.csv` (Permission denied?)

## Get list of all subdirectories in `/data/keeling/a/cristi/a/data/cesm2`

In [5]:
filepathlist = pd.read_csv('/data/keeling/a/cristi/a/esm_data/cesm/subdir_list.csv', 
                       header=None,names=['dirpaths']).values.tolist()

# Transform list of lists to list of strings
filepath=list(itertools.chain.from_iterable(filepathlist))
filepath

['/data/cristi/a/cristi/esm_data/cesm']

## Create Catalog Builder and Build the Catalog

In [6]:
# Create the Builder
cat_builder = Builder(
    # Directory of Organized CMIP6 files 
    paths=filepath)

# Build the Catalog
catalog = cat_builder.build(parsing_func=parse_cesm_timeseries)

/data/keeling/a/mailes2/miniconda3/envs/paleoproj/lib/python3.10/site-packages/ecgtools/builder.py:204: UserWarning: Unable to parse 1 assets. A list of these assets can be found in `.invalid_assets` attribute.
  ).clean_dataframe()


# Check and Save the Catalog

In [8]:
# Browse the catalog
catalog.df

""


In [9]:
# Save the catalog as .csv 
catalog.df.to_csv('/data/keeling/a/cristi/a/esm_data/cesm_lme_catalog.csv', index=False)